# Word Similarity (Constrained)

## Setup (General)

In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv('SimLex-999.txt', delimiter='\t')

df

,word1,word2,POS,SimLex999,conc(w1),conc(w2),concQ,Assoc(USF),SimAssoc333,SD(SimLex)
0,old,new,A,1.58,2.72,2.81,2,7.25,1,0.41
1,smart,intelligent,A,9.20,1.75,2.46,1,7.11,1,0.67
2,hard,difficult,A,8.77,3.76,2.21,2,5.94,1,1.19
3,happy,cheerful,A,9.55,2.56,2.34,1,5.85,1,2.18
4,hard,easy,A,0.95,3.76,2.07,2,5.82,1,0.93
...,...,...,...,...,...,...,...,...,...,...
994,join,acquire,V,2.85,2.86,2.93,2,0.00,0,0.99
995,send,attend,V,1.67,2.70,3.17,2,0.00,0,1.44
996,gather,attend,V,4.80,2.75,3.17,2,0.00,0,1.97
997,absorb,withdraw,V,2.97,3.11,3.04,2,0.00,0,1.75


In [3]:
df.POS.unique()

array(['A', 'N', 'V'], dtype=object)

In [4]:
# Setup for accessing VAD scores
nrc = pd.read_csv('NRC-VAD-Lexicon.txt', delimiter='\t', header=None)

nrc.columns = ['word', 'valence', 'arousal', 'dominance']
nrc = nrc.set_index('word')

nrc

,valence,arousal,dominance
word,,,
aaaaaaah,0.479,0.606,0.291
aaaah,0.520,0.636,0.282
aardvark,0.427,0.490,0.437
aback,0.385,0.407,0.288
abacus,0.510,0.276,0.485
...,...,...,...
zoo,0.760,0.520,0.580
zoological,0.667,0.458,0.492
zoology,0.568,0.347,0.509


In [5]:
nrc.at['bee', 'valence']

0.52

In [6]:
# Store pair-wise SD for each word
pairwise_sd = {}

for _, row in df.iterrows():
  pairwise_sd[row['word1'], row['word2']] = row['SD(SimLex)']

In [7]:
from sklearn.model_selection import train_test_split

# Split the data into training and testing sets (80% train, 20% test)
train, test = train_test_split(df.loc[:, ['word1', 'word2', 'SimLex999']], test_size=0.2, random_state=42)

# Further split the training data into training and validation sets (75% train, 25% validation)
train, val = train_test_split(train, test_size=0.25, random_state=42)


In [8]:
train

,word1,word2,SimLex999
755,meal,waist,0.98
667,belly,abdomen,8.13
704,anger,mood,4.10
608,guy,partner,3.57
605,communication,television,5.60
...,...,...,...
869,get,put,1.98
575,despair,misery,7.22
756,camera,president,0.48
26,happy,angry,1.28


In [9]:
def generate_dataset(df, similarity_functions=None):
  rows = []
  for _, row in df.iterrows():
    word1 = row['word1']
    word2 = row['word2']

    score = row['SimLex999']

    try:
      row = [
              word1,
              word2,
              nrc.at[word1, 'valence'],
              nrc.at[word1, 'arousal'],
              nrc.at[word1, 'dominance'],
              nrc.at[word2, 'valence'],
              nrc.at[word2, 'arousal'],
              nrc.at[word2, 'dominance'],
          ]

      if similarity_functions is not None:
        for func in similarity_functions:
          row.append(func(word1, word2))

      row.append(score)

      rows.append(row)

    except KeyError:
      continue

  return pd.DataFrame(rows)

## Using ONLY VAD

In [10]:
train_VAD = generate_dataset(train)
test_VAD = generate_dataset(test)
val_VAD = generate_dataset(val)

all_VAD = generate_dataset(df.loc[:, ['word1', 'word2', 'SimLex999']])

### SVM

In [11]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score

cross_val_score(LinearRegression(), all_VAD.iloc[:, 2:-1], all_VAD.iloc[:, -1], cv=5).mean()

-0.06451575276478642

In [12]:
from sklearn.svm import SVR

cross_val_score(SVR(), all_VAD.iloc[:, 2:-1], all_VAD.iloc[:, -1], cv=5).mean()

0.08234493656036967

In [13]:
from sklearn.ensemble import RandomForestRegressor

cross_val_score(RandomForestRegressor(n_estimators=100), all_VAD.iloc[:, 2:-1], all_VAD.iloc[:, -1], cv=5).mean()

0.047966431780471705

In [14]:
svr_VAD = SVR()

svr_VAD.fit(train_VAD.iloc[:, 2:-1], train_VAD.iloc[:, -1])

SVR()

In [19]:
from sklearn.metrics import mean_squared_error

y_pred = svr_VAD.predict(test_VAD.iloc[:, 2:-1])
y_test = test_VAD.iloc[:, -1].values

mean_squared_error(y_test, y_pred)

5.264556766795256

In [26]:
def report_performance(df, y_pred, y_test):
  df4 = pd.DataFrame(list(zip(y_test, y_pred)), columns=['True', 'Pred'])

  df4['word1'] = df.iloc[:, 0]
  df4['word2'] = df.iloc[:, 1]

  df4['abs_error'] = (df4['True'] - df4['Pred']).apply(lambda x: abs(x))

  SD_ratio = []
  for _, row in df4.iterrows():
    SD_ratio.append(row['abs_error'] / pairwise_sd[(row.word1, row.word2)])

  df4['SD_ratio'] = np.array(SD_ratio)

  return {
      '<=1': len(df4[df4['SD_ratio'].apply(lambda x: x<=1)]),
      '<=2': len(df4[df4['SD_ratio'].apply(lambda x: x<=2)])
  }

In [27]:
report_performance(test_VAD, y_test, y_pred)

{'<=1': 78, '<=2': 134}

In [28]:
report = {}

In [29]:
report['SVM_VAD'] = report_performance(test_VAD, y_test, y_pred)

In [31]:
report_df = pd.DataFrame.from_dict(report, orient='index')
report_df

,<=1,<=2
SVM_VAD,78,134


### NN

In [34]:
X_train = train_VAD.iloc[:, :-1].values
y_train = train_VAD.iloc[:, -1].values

X_test = test_VAD.iloc[:, :-1].values
y_test = test_VAD.iloc[:, -1].values

X_val = val_VAD.iloc[:, :-1].values
y_val = val_VAD.iloc[:, -1].values

In [35]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models


model = models.Sequential([
    layers.Dense(128, activation='relu', input_shape=(6,)),
    layers.Dense(64, activation='relu'),
    layers.Dense(1)
])

model.compile(optimizer='adam',
              loss='mean_squared_error',  # Use Mean Squared Error for regression
              metrics=['mean_squared_error'])  # Use Mean Absolute Error as metric

model.fit(X_train[:, 2:].astype(np.float64), y_train,
          epochs=100, batch_size=16,
          validation_data=(X_val[:, 2:].astype(np.float64), y_val))

test_loss, test_mse = model.evaluate(X_test[:, 2:].astype(np.float64), y_test)
print('Test Mean Squared Error:', test_mse)

Epoch 1/100
36/36 [==============================] - 2s 10ms/step - loss: 15.4217 - mean_squared_error: 15.4217 - val_loss: 7.3787 - val_mean_squared_error: 7.3787
Epoch 2/100
36/36 [==============================] - 0s 3ms/step - loss: 6.8867 - mean_squared_error: 6.8867 - val_loss: 7.3488 - val_mean_squared_error: 7.3488
Epoch 3/100
36/36 [==============================] - 0s 3ms/step - loss: 6.7210 - mean_squared_error: 6.7210 - val_loss: 6.8348 - val_mean_squared_error: 6.8348
Epoch 4/100
36/36 [==============================] - 0s 3ms/step - loss: 6.6128 - mean_squared_error: 6.6128 - val_loss: 6.6702 - val_mean_squared_error: 6.6702
Epoch 5/100
36/36 [==============================] - 0s 3ms/step - loss: 6.5909 - mean_squared_error: 6.5909 - val_loss: 6.8982 - val_mean_squared_error: 6.8982
Epoch 6/100
36/36 [==============================] - 0s 3ms/step - loss: 6.4683 - mean_squared_error: 6.4683 - val_loss: 6.5447 - val_mean_squared_error: 6.5447
Epoch 7/100
36/36 [============

In [37]:
y_pred = model.predict(X_test[:, 2:].astype(np.float64))

7/7 [==============================] - 0s 4ms/step


In [39]:
report_performance(test_VAD, y_test, y_pred)

{'<=1': 80, '<=2': 130}

## VAD + Word2Vec

### Setup

In [40]:
import nltk
nltk.download('brown')

from nltk.corpus import brown

[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.


In [41]:
import gensim
from gensim.models import Word2Vec

w2v = Word2Vec(brown.sents(), min_count=1, vector_size=100, window=5)

In [42]:
def wordvec_similarity(word1, word2):
  return w2v.wv.similarity(word1, word2)

In [44]:
train_VAD_WV = generate_dataset(train, [wordvec_similarity])
test_VAD_WV = generate_dataset(test, [wordvec_similarity])
val_VAD_WV = generate_dataset(val, [wordvec_similarity])


X_train = train_VAD_WV.iloc[:, :-1].values
y_train = train_VAD_WV.iloc[:, -1].values

X_test = test_VAD_WV.iloc[:, :-1].values
y_test = test_VAD_WV.iloc[:, -1].values

X_val = val_VAD_WV.iloc[:, :-1].values
y_val = val_VAD_WV.iloc[:, -1].values

In [45]:
svr_VAD_WV = SVR()

svr_VAD_WV.fit(X_train[:, 2:].astype(np.float64), y_train)

SVR()

In [46]:
y_pred = svr_VAD_WV.predict(X_test[:, 2:].astype(np.float64))

mean_squared_error(y_test, y_pred)

# 5.53500159519488

5.331635730757275

In [47]:
report_performance(test_VAD_WV, y_test, y_pred)

{'<=1': 80, '<=2': 130}

### NN

In [49]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models


model = models.Sequential([
    layers.Dense(128, activation='relu', input_shape=(7,)),
    layers.Dense(64, activation='relu'),
    layers.Dense(1)
])

model.compile(optimizer='adam',
              loss='mean_squared_error',  # Use Mean Squared Error for regression
              metrics=['mean_squared_error'])  # Use Mean Absolute Error as metric

model.fit(X_train[:, 2:].astype(np.float64), y_train,
          epochs=100, batch_size=16,
          validation_data=(X_val[:, 2:].astype(np.float64), y_val))

test_loss, test_mse = model.evaluate(X_test[:, 2:].astype(np.float64), y_test)
print('Test Mean Squared Error:', test_mse)

Epoch 1/100
36/36 [==============================] - 3s 30ms/step - loss: 17.5167 - mean_squared_error: 17.5167 - val_loss: 8.8648 - val_mean_squared_error: 8.8648
Epoch 2/100
36/36 [==============================] - 0s 7ms/step - loss: 6.7426 - mean_squared_error: 6.7426 - val_loss: 6.8069 - val_mean_squared_error: 6.8069
Epoch 3/100
36/36 [==============================] - 0s 5ms/step - loss: 6.5927 - mean_squared_error: 6.5927 - val_loss: 6.7757 - val_mean_squared_error: 6.7757
Epoch 4/100
36/36 [==============================] - 0s 5ms/step - loss: 6.5477 - mean_squared_error: 6.5477 - val_loss: 6.7611 - val_mean_squared_error: 6.7611
Epoch 5/100
36/36 [==============================] - 0s 6ms/step - loss: 6.4780 - mean_squared_error: 6.4780 - val_loss: 6.6379 - val_mean_squared_error: 6.6379
Epoch 6/100
36/36 [==============================] - 0s 5ms/step - loss: 6.4541 - mean_squared_error: 6.4541 - val_loss: 6.6566 - val_mean_squared_error: 6.6566
Epoch 7/100
36/36 [============

In [50]:
y_pred = model.predict(X_test[:, 2:].astype(np.float64))

report_performance(test_VAD_WV, y_test, y_pred)

7/7 [==============================] - 0s 2ms/step


{'<=1': 71, '<=2': 129}

## WordNet

In [51]:
import nltk
nltk.download('wordnet')

from nltk.corpus import wordnet

def word_similarity(word1, word2):
    # Get synsets for each word
    synsets1 = wordnet.synsets(word1)
    synsets2 = wordnet.synsets(word2)

    max_similarity = 0.0

    for synset1 in synsets1:
        for synset2 in synsets2:
            # Calculate similarity between synsets
            similarity = synset1.wup_similarity(synset2)
            if similarity is not None and similarity > max_similarity:
                max_similarity = similarity

    return max_similarity

word1 = "agreement"
word2 = "argument"
similarity_score = word_similarity(word1, word2)
print(f"Similarity between '{word1}' and '{word2}': {similarity_score}")


[nltk_data] Downloading package wordnet to /root/nltk_data...


Similarity between 'agreement' and 'argument': 0.7777777777777778


In [52]:
train_VAD_WN = generate_dataset(train, [word_similarity])
test_VAD_WN = generate_dataset(test, [word_similarity])
val_VAD_WN = generate_dataset(val, [word_similarity])


X_train = train_VAD_WN.iloc[:, :-1].values
y_train = train_VAD_WN.iloc[:, -1].values

X_test = test_VAD_WN.iloc[:, :-1].values
y_test = test_VAD_WN.iloc[:, -1].values

X_val = val_VAD_WN.iloc[:, :-1].values
y_val = val_VAD_WN.iloc[:, -1].values

### SVM

In [53]:
svr_VAD_WN = SVR()

svr_VAD_WN.fit(X_train[:, 2:].astype(np.float64), y_train)

SVR()

In [55]:
y_pred = svr_VAD_WN.predict(X_test[:, 2:].astype(np.float64))

mean_squared_error(y_test, y_pred)


4.078742705415954

In [56]:
report_performance(test_VAD_WN, y_test, y_pred)

{'<=1': 89, '<=2': 151}

### NN

In [57]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models


model = models.Sequential([
    layers.Dense(128, activation='relu', input_shape=(7,)),
    layers.Dense(64, activation='relu'),
    layers.Dense(1)
])

model.compile(optimizer='adam',
              loss='mean_squared_error',  # Use Mean Squared Error for regression
              metrics=['mean_squared_error'])  # Use Mean Absolute Error as metric

model.fit(X_train[:, 2:].astype(np.float64), y_train,
          epochs=100, batch_size=16,
          validation_data=(X_val[:, 2:].astype(np.float64), y_val))

test_loss, test_mse = model.evaluate(X_test[:, 2:].astype(np.float64), y_test)
print('Test Mean Squared Error:', test_mse)

Epoch 1/100
36/36 [==============================] - 1s 7ms/step - loss: 16.3666 - mean_squared_error: 16.3666 - val_loss: 8.4015 - val_mean_squared_error: 8.4015
Epoch 2/100
36/36 [==============================] - 0s 3ms/step - loss: 6.5304 - mean_squared_error: 6.5304 - val_loss: 6.3095 - val_mean_squared_error: 6.3095
Epoch 3/100
36/36 [==============================] - 0s 3ms/step - loss: 6.0188 - mean_squared_error: 6.0188 - val_loss: 6.1990 - val_mean_squared_error: 6.1990
Epoch 4/100
36/36 [==============================] - 0s 3ms/step - loss: 5.8852 - mean_squared_error: 5.8852 - val_loss: 6.1350 - val_mean_squared_error: 6.1350
Epoch 5/100
36/36 [==============================] - 0s 4ms/step - loss: 5.7023 - mean_squared_error: 5.7023 - val_loss: 5.6819 - val_mean_squared_error: 5.6819
Epoch 6/100
36/36 [==============================] - 0s 6ms/step - loss: 5.5163 - mean_squared_error: 5.5163 - val_loss: 5.4281 - val_mean_squared_error: 5.4281
Epoch 7/100
36/36 [=============

In [58]:
y_pred = model.predict(X_test[:, 2:].astype(np.float64))

report_performance(test_VAD_WN, y_test, y_pred)

7/7 [==============================] - 0s 3ms/step


{'<=1': 89, '<=2': 144}

## All Combined

In [60]:
train_ALL = generate_dataset(train, [word_similarity, wordvec_similarity])
test_ALL = generate_dataset(test, [word_similarity, wordvec_similarity])
val_ALL = generate_dataset(val, [word_similarity, wordvec_similarity])


X_train = train_ALL.iloc[:, :-1].values
y_train = train_ALL.iloc[:, -1].values

X_test = test_ALL.iloc[:, :-1].values
y_test = test_ALL.iloc[:, -1].values

X_val = val_ALL.iloc[:, :-1].values
y_val = val_ALL.iloc[:, -1].values

### SVM

In [61]:
svr_ALL = SVR()

svr_ALL.fit(X_train[:, 2:].astype(np.float64), y_train)

SVR()

In [63]:
y_pred = svr_ALL.predict(X_test[:, 2:].astype(np.float64))

mean_squared_error(y_test, y_pred)

# 5.53500159519488

4.048851009062132

In [64]:
report_performance(test_ALL, y_test, y_pred)

{'<=1': 91, '<=2': 152}

### NN

In [65]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models


model = models.Sequential([
    layers.Dense(128, activation='relu', input_shape=(8,)),
    layers.Dense(64, activation='relu'),
    layers.Dense(1)
])

model.compile(optimizer='adam',
              loss='mean_squared_error',  # Use Mean Squared Error for regression
              metrics=['mean_squared_error'])  # Use Mean Absolute Error as metric

model.fit(X_train[:, 2:].astype(np.float64), y_train,
          epochs=100, batch_size=16,
          validation_data=(X_val[:, 2:].astype(np.float64), y_val))

test_loss, test_mse = model.evaluate(X_test[:, 2:].astype(np.float64), y_test)
print('Test Mean Squared Error:', test_mse)

Epoch 1/100
36/36 [==============================] - 2s 17ms/step - loss: 17.1659 - mean_squared_error: 17.1659 - val_loss: 7.2218 - val_mean_squared_error: 7.2218
Epoch 2/100
36/36 [==============================] - 0s 7ms/step - loss: 6.3467 - mean_squared_error: 6.3467 - val_loss: 6.2572 - val_mean_squared_error: 6.2572
Epoch 3/100
36/36 [==============================] - 0s 7ms/step - loss: 5.9580 - mean_squared_error: 5.9580 - val_loss: 5.7545 - val_mean_squared_error: 5.7545
Epoch 4/100
36/36 [==============================] - 0s 9ms/step - loss: 5.8843 - mean_squared_error: 5.8843 - val_loss: 5.5850 - val_mean_squared_error: 5.5850
Epoch 5/100
36/36 [==============================] - 0s 6ms/step - loss: 5.7327 - mean_squared_error: 5.7327 - val_loss: 5.4757 - val_mean_squared_error: 5.4757
Epoch 6/100
36/36 [==============================] - 0s 4ms/step - loss: 5.5045 - mean_squared_error: 5.5045 - val_loss: 5.4161 - val_mean_squared_error: 5.4161
Epoch 7/100
36/36 [============

In [66]:
y_pred = model.predict(X_test[:, 2:].astype(np.float64))

report_performance(test_ALL, y_test, y_pred)

7/7 [==============================] - 0s 2ms/step


{'<=1': 88, '<=2': 145}

In [69]:
w2v.wv['quick'].shape

(100,)

In [72]:
v1 = list(w2v.wv['quick'])
v2 = list(w2v.wv['rapid'])


np.dot(v1, v2) / ((np.dot(v1, v1) * np.dot(v2, v2)) ** 0.5)

0.881017730225204

In [71]:
w2v.wv.similarity('quick', 'rapid')

0.8810177

In [73]:
v1_ = v1 + [nrc.at['quick', 'valence'], nrc.at['quick', 'arousal'],
            nrc.at['quick', 'dominance']]
v2_ = v2 + [nrc.at['rapid', 'valence'], nrc.at['rapid', 'arousal'],
            nrc.at['rapid', 'dominance']]

In [74]:
np.dot(v1_, v2_)

5.909096714668699

In [75]:
np.dot(v1_, v2_) / ((np.dot(v1_, v1_) * np.dot(v2_, v2_)) ** 0.5)

0.9081777548879469

In [76]:
def sim(w1, w2):
  v1 = list(w2v.wv[w1])
  v2 = list(w2v.wv[w2])

  v1_ = v1 + [nrc.at[w1, 'valence'], nrc.at[w1, 'arousal'],
            nrc.at[w1, 'dominance']]
  v2_ = v2 + [nrc.at[w2, 'valence'], nrc.at[w2, 'arousal'],
            nrc.at[w2, 'dominance']]
  return np.dot(v1_, v2_) / ((np.dot(v1_, v1_) * np.dot(v2_, v2_)) ** 0.5)

In [78]:
sim('quick', 'rapid')

0.9081777548879469

In [79]:
sim('sharp', 'dull')

0.9233064857658595

In [81]:
def sim_(w1, w2):
  v1_ = list(w2v.wv[w1])
  v2_ = list(w2v.wv[w2])

  # v1_ = v1 + [nrc.at[w1, 'valence'], nrc.at[w1, 'arousal'],
  #          nrc.at[w1, 'dominance']]
  # v2_ = v2 + [nrc.at[w2, 'valence'], nrc.at[w2, 'arousal'],
  #          nrc.at[w2, 'dominance']]
  return np.dot(v1_, v2_) / ((np.dot(v1_, v1_) * np.dot(v2_, v2_)) ** 0.5)

In [82]:
sim_('sharp', 'dull')

0.9250461166623755